In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/buses/bangalore-bus-tickets"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def close_pop_up(driver):
    try:
        # Assume pop-up has a close button with a certain class or id
        pop_up_close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='close']"))
        )
        pop_up_close_button.click()
        time.sleep(1)  # Give it some time to close
    except Exception as e:
        # If no pop-up, or pop-up couldn't be closed, just pass
        pass


# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        close_pop_up(driver)
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 3):  # There are 2 pages
        try:
            load_page(driver, URL)
            
            if page >= 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('bangalore_bus_details.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


In [5]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,Jabbar Travels,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),23:40,06h 05m,05:45,4.2,INR 500,26 Seats available
1,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,SPT Tours and Travels,A/C Sleeper (2+1),22:05,07h 20m,05:25,4.3,648,18 Seats available
2,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,,,,,,,,
3,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,,,,,,,,
4,Chennai to Bangalore,https://www.redbus.in/bus-tickets/chennai-to-b...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1374,Bangalore to Tiruchirapalli,https://www.redbus.in/bus-tickets/bangalore-to...,Sri Renganathan Travels,A/C Sleeper (2+1),13:30,07h 45m,21:15,4.0,INR 650,15 Seats available
1375,Bangalore to Tiruchirapalli,https://www.redbus.in/bus-tickets/bangalore-to...,Rithish Travels,,21:45,07h 15m,05:00,4.0,569,
1376,Bangalore to Tiruchirapalli,https://www.redbus.in/bus-tickets/bangalore-to...,,,,,,,,
1377,Bangalore to Tiruchirapalli,https://www.redbus.in/bus-tickets/bangalore-to...,,,,,,,,
